In [19]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

def compare_performance(json_file_cpu, json_file_1gpu, json_file_2gpus, save_dir='plots'):
    """
    Compare performance metrics (speedup, throughput, losses) across CPU, 1 GPU, and 2 GPUs (Rank 0, Rank 1).
    Save all plots to a specified directory.
    """
    # Create directory to save plots if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Load CPU data
    with open(json_file_cpu, 'r') as f:
        data_cpu = json.load(f)
    seq2seq_cpu = data_cpu['model_vgg11_epochs_3_1_cpu_rank_0']

    # Load 1-GPU data
    with open(json_file_1gpu, 'r') as f:
        data_1gpu = json.load(f)
    seq2seq_1gpu = data_1gpu['model_vgg11_epochs_3_1_cuda_rank_0']

    # Load 2-GPU data
    with open(json_file_2gpus, 'r') as f:
        data_2gpus = json.load(f)
    seq2seq_2gpus_rank0 = data_2gpus['model_vgg11_epochs_3_2_cuda_rank_0']
    seq2seq_2gpus_rank1 = data_2gpus['model_vgg11_epochs_3_2_cuda_rank_1']

    # Extract data
    train_times_cpu = seq2seq_cpu['training_times']
    train_times_1gpu = seq2seq_1gpu['training_times']
    train_times_2gpus_rank0 = seq2seq_2gpus_rank0['training_times']
    train_times_2gpus_rank1 = seq2seq_2gpus_rank1['training_times']
    train_times_2gpus_avg = np.mean([train_times_2gpus_rank0, train_times_2gpus_rank1], axis=0)

    train_losses_cpu = seq2seq_cpu['train_losses']
    train_losses_1gpu = seq2seq_1gpu['train_losses']
    train_losses_2gpus_rank0 = seq2seq_2gpus_rank0['train_losses']
    train_losses_2gpus_rank1 = seq2seq_2gpus_rank1['train_losses']
    train_losses_2gpus_avg = np.mean([train_losses_2gpus_rank0, train_losses_2gpus_rank1], axis=0)

    train_throughput_cpu = seq2seq_cpu['train_throughputs']
    train_throughput_1gpu = seq2seq_1gpu['train_throughputs']
    train_throughput_2gpus_rank0 = seq2seq_2gpus_rank0['train_throughputs']
    train_throughput_2gpus_rank1 = seq2seq_2gpus_rank1['train_throughputs']
    train_throughputs_2gpus_total = np.array(train_throughput_2gpus_rank0) + np.array(train_throughput_2gpus_rank1)

    # Calculate speedup
    speedup_1gpu = np.array(train_times_cpu) / np.array(train_times_1gpu)
    speedup_2gpus_rank0 = np.array(train_times_cpu) / np.array(train_times_2gpus_rank0)
    speedup_2gpus_rank1 = np.array(train_times_cpu) / np.array(train_times_2gpus_rank1)
    speedup_2gpus_avg = np.array(train_times_cpu) / np.array(train_times_2gpus_avg)

    epochs = np.arange(1, len(train_times_cpu) + 1)

    # Function to save plots
    def save_plot(plt, filename):
        """Helper function to save plots."""
        plt.savefig(os.path.join(save_dir, filename), bbox_inches='tight')
        plt.close()

    # Plot speedup
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, speedup_1gpu, label="1 GPU Speedup", marker='o', color='blue')
    plt.plot(epochs, speedup_2gpus_rank0, label="2 GPUs Speedup (Rank 0)", marker='o', color='orange')
    plt.plot(epochs, speedup_2gpus_rank1, label="2 GPUs Speedup (Rank 1)", marker='o', color='green')
    plt.plot(epochs, speedup_2gpus_avg, label="2 GPUs Speedup (Avg)", marker='o', color='red')
    plt.xlabel("Epochs")
    plt.ylabel("Speedup")
    plt.title("Speedup Comparison")
    plt.legend()
    plt.grid()
    save_plot(plt, 'speedup_comparison.png')

    # Plot throughput
    plt.figure(figsize=(10, 6))
    plt.bar(['CPU', '1 GPU', '2 GPUs (Rank 0)', '2 GPUs (Rank 1)', '2 GPUs (Total)'], [
        np.mean(train_throughput_cpu),
        np.mean(train_throughput_1gpu),
        np.mean(train_throughput_2gpus_rank0),
        np.mean(train_throughput_2gpus_rank1),
        np.mean(train_throughputs_2gpus_total)
    ], color=['blue', 'green', 'orange', 'red', 'purple'])
    plt.ylabel("Throughput (samples/s)")
    plt.title("Throughput Comparison")
    save_plot(plt, 'throughput_comparison.png')

    # Plot training loss
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses_cpu, label="CPU", marker='o', color='blue')
    plt.plot(epochs, train_losses_1gpu, label="1 GPU", marker='o', color='green')
    plt.plot(epochs, train_losses_2gpus_rank0, label="2 GPUs (Rank 0)", marker='o', color='orange')
    plt.plot(epochs, train_losses_2gpus_rank1, label="2 GPUs (Rank 1)", marker='o', color='red')
    plt.plot(epochs, train_losses_2gpus_avg, label="2 GPUs (Avg)", marker='o', color='purple')
    plt.xlabel("Epochs")
    plt.ylabel("Training Loss")
    plt.title("Training Loss Comparison")
    plt.legend()
    plt.grid()
    save_plot(plt, 'training_loss_comparison.png')

if __name__ == "__main__":
    compare_performance(
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3-cpu.json',
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3_1gpu.json',
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3_2gpus.json',
        save_dir='plots'  # Directory to save plots
    )

In [18]:
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

def compare_performance(json_file_cpu, json_file_1gpu, json_file_2gpus, save_dir='plots'):
    """
    Compare performance metrics (speedup, throughput, losses, accuracies) across CPU, 1 GPU, and 2 GPUs (Rank 0, Rank 1).
    Includes confusion matrix visualization and saves all plots to a specified directory.
    """
    # Create directory to save plots if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Load CPU data
    with open(json_file_cpu, 'r') as f:
        data_cpu = json.load(f)
    seq2seq_cpu = data_cpu['model_vgg11_epochs_3_1_cpu_rank_0']

    # Load 1-GPU data
    with open(json_file_1gpu, 'r') as f:
        data_1gpu = json.load(f)
    seq2seq_1gpu = data_1gpu['model_vgg11_epochs_3_1_cuda_rank_0']

    # Load 2-GPU data
    with open(json_file_2gpus, 'r') as f:
        data_2gpus = json.load(f)
    seq2seq_2gpus_rank0 = data_2gpus['model_vgg11_epochs_3_2_cuda_rank_0']
    seq2seq_2gpus_rank1 = data_2gpus['model_vgg11_epochs_3_2_cuda_rank_1']

    # Calculate average metrics for 2 GPUs
    seq2seq_2gpus_avg = {
        'training_times': np.mean([seq2seq_2gpus_rank0['training_times'], seq2seq_2gpus_rank1['training_times']], axis=0),
        'train_losses': np.mean([seq2seq_2gpus_rank0['train_losses'], seq2seq_2gpus_rank1['train_losses']], axis=0),
        'train_accurcy': np.mean([seq2seq_2gpus_rank0['train_accurcy'], seq2seq_2gpus_rank1['train_accurcy']], axis=0),
        'validation_times': np.mean([seq2seq_2gpus_rank0['validation_times'], seq2seq_2gpus_rank1['validation_times']], axis=0),
        'validation_losses': np.mean([seq2seq_2gpus_rank0['validation_losses'], seq2seq_2gpus_rank1['validation_losses']], axis=0),
        'validation_accurcy': np.mean([seq2seq_2gpus_rank0['validation_accurcy'], seq2seq_2gpus_rank1['validation_accurcy']], axis=0),
        'test_time': np.mean([seq2seq_2gpus_rank0['test_time'], seq2seq_2gpus_rank1['test_time']]),
        'test_loss': np.mean([seq2seq_2gpus_rank0['test_loss'], seq2seq_2gpus_rank1['test_loss']]),
        'test_acc': np.mean([seq2seq_2gpus_rank0['test_acc'], seq2seq_2gpus_rank1['test_acc']]),
        'epoch_times': np.mean([seq2seq_2gpus_rank0['epoch_times'], seq2seq_2gpus_rank1['epoch_times']], axis=0),
        'train_throughputs': np.mean([seq2seq_2gpus_rank0['train_throughputs'], seq2seq_2gpus_rank1['train_throughputs']], axis=0),
        'validation_throughputs': np.mean([seq2seq_2gpus_rank0['validation_throughputs'], seq2seq_2gpus_rank1['validation_throughputs']], axis=0),
        'test_throughput': np.mean([seq2seq_2gpus_rank0['test_throughput'], seq2seq_2gpus_rank1['test_throughput']]),
        'confusion_matrix': np.mean([seq2seq_2gpus_rank0['confusion_matrix'], seq2seq_2gpus_rank1['confusion_matrix']], axis=0),
    }

    # Plotting
    epochs = range(1, len(seq2seq_cpu['training_times']) + 1)

    # Function to save plots
    def save_plot(plt, filename):
        """Helper function to save plots."""
        plt.savefig(os.path.join(save_dir, filename), bbox_inches='tight')
        plt.close()

    # Plot Training Times
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['training_times'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['training_times'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['training_times'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['training_times'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['training_times'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Training Time (s)')
    plt.title('Training Time Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'training_time_comparison.png')

    # Plot Validation Times
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['validation_times'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['validation_times'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['validation_times'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['validation_times'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['validation_times'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Time (s)')
    plt.title('Validation Time Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'validation_time_comparison.png')

    # Plot Training Losses
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['train_losses'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['train_losses'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['train_losses'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['train_losses'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['train_losses'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Training Loss')
    plt.title('Training Loss Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'training_loss_comparison.png')

    # Plot Training Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['train_accurcy'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['train_accurcy'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['train_accurcy'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['train_accurcy'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['train_accurcy'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Training Accuracy (%)')
    plt.title('Training Accuracy Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'training_accuracy_comparison.png')

    # Plot Validation Losses
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['validation_losses'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['validation_losses'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['validation_losses'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['validation_losses'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['validation_losses'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.title('Validation Loss Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'validation_loss_comparison.png')

    # Plot Validation Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['validation_accurcy'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['validation_accurcy'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['validation_accurcy'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['validation_accurcy'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['validation_accurcy'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Accuracy (%)')
    plt.title('Validation Accuracy Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'validation_accuracy_comparison.png')

    # Plot Training Throughput
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, seq2seq_cpu['train_throughputs'], label='CPU', marker='o')
    plt.plot(epochs, seq2seq_1gpu['train_throughputs'], label='1 GPU', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank0['train_throughputs'], label='2 GPUs (Rank 0)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_rank1['train_throughputs'], label='2 GPUs (Rank 1)', marker='o')
    plt.plot(epochs, seq2seq_2gpus_avg['train_throughputs'], label='2 GPUs (Average)', marker='o', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Training Throughput (samples/s)')
    plt.title('Training Throughput Comparison')
    plt.legend()
    plt.grid(True)
    save_plot(plt, 'training_throughput_comparison.png')

    # Plot Test Accuracy
    plt.figure(figsize=(10, 6))
    plt.bar(['CPU', '1 GPU', '2 GPUs (Avg)'], [seq2seq_cpu['test_acc'], seq2seq_1gpu['test_acc'], seq2seq_2gpus_avg['test_acc']])
    plt.xlabel('Device')
    plt.ylabel('Test Accuracy (%)')
    plt.title('Test Accuracy Comparison')
    plt.grid(True)
    save_plot(plt, 'test_accuracy_comparison.png')

    # Plot Confusion Matrix for 1 GPU
    plt.figure(figsize=(10, 8))
    confusion_matrix_1gpu = np.array(seq2seq_1gpu['confusion_matrix']).astype(int)
    sns.heatmap(confusion_matrix_1gpu, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix (1 GPU)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    save_plot(plt, 'confusion_matrix_1gpu.png')

    # Plot Confusion Matrix for 2 GPUs (Average)
    plt.figure(figsize=(10, 8))
    confusion_matrix_2gpus_avg = np.array(seq2seq_2gpus_avg['confusion_matrix']).astype(int)
    sns.heatmap(confusion_matrix_2gpus_avg, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix (2 GPUs Avg)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    save_plot(plt, 'confusion_matrix_2gpus_avg.png')

    # Plot Confusion Matrix for CPU
    plt.figure(figsize=(10, 8))
    confusion_matrix_cpu = np.array(seq2seq_cpu['confusion_matrix']).astype(int)
    sns.heatmap(confusion_matrix_cpu, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix (CPU)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    save_plot(plt, 'confusion_matrix_cpu.png')

if __name__ == "__main__":
    compare_performance(
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3-cpu.json',
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3_1gpu.json',
        r'C:\Users\hafss\OneDrive\Desktop\GPU\project3\project3_2gpus.json',
        save_dir='plots'  # Directory to save plots
    )